<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Combining flats


In [ ]:
from pathlib import Path

import numpy as np

import ccdproc as ccdp
from astropy.stats import mad_std

from convenience_functions import show_image

In [ ]:
calibrated_path = Path('example1-reduced')

ifc = ccdp.ImageFileCollection(p)

In [ ]:
flat_filters = set(h['filter'] for h in ifc.headers(imagetyp='flatfield'))
flat_filters

In [ ]:
def inv_median(a):
    return 1 / np.median(a)

for filt in flat_filters:
    to_combine = ifc.files_filtered(imagetyp='flatfield', filter=filt, include_path=True)
    combined_flat = ccdp.combine(to_combine,
                                 method='average', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                 mem_limit=350e6
                                )

    combined_flat.meta['combined'] = True
    dark_file_name = 'combined_flat_filter_{}.fit'.format(filt.replace("''", "p"))
    combined_flat.write(calibrated_path / dark_file_name)

In [ ]:
show_image(combined_flat, cmap='gray')